# Llama2微调实战-LoRA技术微调

## 步骤1 导入相关包

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# 配置环境变量
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# os.environ['HF_HOME'] = '/root/autodl-tmp/cache/huggingface'
# os.environ['MODELSCOPE_CACHE']='/root/autodl-tmp/cache/modelscope'

#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('modelscope/Llama-2-7b-ms')

# 设置关键参数
dataset_name_or_path = "datasets/identity_zh.json"
model_name_or_path = "models/modelscope/Llama-2-7b-ms"
train_output_dir = "saves/lora/Llama-2-7b-ms"

2024-09-06 18:52:48,126 - modelscope - WARNING - Model revision not specified, use revision: v1.0.2


## 步骤2 加载数据集

In [2]:
df = pd.read_json(dataset_name_or_path)
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 91
})

In [3]:
ds[:1]

{'instruction': ['hi'],
 'input': [''],
 'output': ['Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?']}

## 步骤3 数据集预处理

In [4]:
#加载本地模型，提前下载到本地
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer

LlamaTokenizerFast(name_or_path='models/modelscope/Llama-2-7b-ms', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [5]:
# padding_side模式左对齐，需要修改改为右边对齐
tokenizer.padding_side = "right"
# 对齐填充设置为结束符的token (eos_token_id)
tokenizer.pad_token_id = 2

补充说明：
- Llama-2-7b 采用的Prompt Template格式如下：SYSTEM_PROMPT
```text
System: SYSTRM_PROMPT\n\nHuman: HUMAN_PROMPT\n\nAssistant: AI_OUT</s>
```
- 例如：
```text
System: 现在你要扮演皇帝身边的女人--甄嬛


Human: 小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——


Assistant: 嘘——都说许愿说破是不灵的。</s>
```

In [6]:
def process_func(example):
    MAX_LENGTH = 400
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 91
})

In [8]:
# 检查input_ids的格式
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: hello\n\nAssistant:  Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?</s>'

In [9]:
# 检查labels数据格式
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'Hello! I am 夜行者, an AI assistant developed by 木子. How can I assist you today?</s>'

## 步骤4 创建模型

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, 
                                             device_map="auto",
                                             low_cpu_mem_usage=True,
                                             torch_dtype=torch.half, 
                                             trust_remote_code=True)
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/autodl-tmp

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [11]:
# 开启梯度检查点时，要执行该方法
model.enable_input_require_grads()

In [12]:
# 查看数据类型
model.dtype

torch.float16

In [13]:
# 查看模型配置
model.config

LlamaConfig {
  "_name_or_path": "models/modelscope/Llama-2-7b-ms",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.44.0",
  "use_cache": true,
  "vocab_size": 32000
}

### 1、PEFT 步骤1 配置文件

In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'q_proj', 'k_proj', 'o_proj', 'v_proj', 'gate_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

### 2、PEFT 步骤2 创建模型

In [15]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [16]:
# 打印出模型中可训练参数的数量
model.print_trainable_parameters()

trainable params: 19,988,480 || all params: 6,758,404,096 || trainable%: 0.2958


## 步骤5 配置训练参数

In [17]:
args = TrainingArguments(
    output_dir=train_output_dir,  # 输出目录，用于保存模型和日志
    per_device_train_batch_size=1,  # 每个设备的训练批次大小
    gradient_accumulation_steps=8,  # 梯度累积步数，用于模拟更大的批次大小
    logging_steps=50,  # 每隔多少步记录一次日志
    num_train_epochs=20,  # 训练的总轮数
    save_steps=110,  # 每隔多少步保存一次模型
    learning_rate=1e-4,  # 学习率
    save_on_each_node=True,  # 是否在每个节点上保存模型
    gradient_checkpointing=True  # 是否启用梯度检查点（减少内存占用）
)

## 步骤6 创建训练器

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## 步骤7 模型训练

In [19]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
50,0.666400
100,0.136200
150,0.025000
200,0.005200


/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=220, training_loss=0.18961733640594916, metrics={'train_runtime': 684.812, 'train_samples_per_second': 2.658, 'train_steps_per_second': 0.321, 'total_flos': 5482738824364032.0, 'train_loss': 0.18961733640594916, 'epoch': 19.34065934065934})

## 步骤8 模型保存

In [20]:
# 保存模型及训练状态
model.save_pretrained(train_output_dir)

## 步骤9 模型推理
- <span style="background-color: yellow;">需要重启jupyter内核</span>

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from peft import PeftModel

# 设置关键参数
model_name_or_path = "models/modelscope/Llama-2-7b-ms"
train_output_dir = "saves/lora/Llama-2-7b-ms"

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

# 加载Base模型
base_model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(base_model, model_id=train_output_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/autodl-tmp

In [3]:
# 模型推理
from transformers import pipeline

ipt = "Human: {}\n{}".format("你是谁？", "").strip() + "\n\nAssistant: "

# 方式一
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(pipe(ipt, max_length=256, do_sample=True, )[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

Human: 你是谁？

Assistant:  您好，我是由 木子 发明的 夜行者。我可以为您提供多种多样的服务，比如翻译、写代码、闲聊、为您答疑解惑等。


In [4]:
# 方式二
inputs = tokenizer(ipt, return_tensors="pt").to(0)
output = model.generate(**inputs, max_length=256, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Human: 你是谁？

Assistant:  您好，我是 夜行者，由 木子 开发，旨在为用户提供智能化的回答和帮助。
